## 1.将对原始数据预处理完成后保存的CSV文件读入

In [45]:
import pandas as pd
path = './input/'
datasc = pd.read_csv(path + '2.课程测验成绩表.csv')
datacy = pd.read_csv(path + '3.学生课堂活动参与表.csv')
datatl = pd.read_csv(path + '4.学生课堂讨论流水表.csv')
datarw = pd.read_csv(path + '5.课程任务点完成记录表_1.csv')
datals = pd.read_csv(path + '6.其他活动流水和登录流水.csv')
datazy = pd.read_csv(path + '7.学生课程作业成绩表_1.csv')
dfscore = pd.read_csv(path + '8.学生综合考试成绩表.csv')

### 汇总数据
492504行4列的数据，将其中“活动类型”列的取值作为特征维度

In [41]:
data = pd.concat([datacy[datacy.columns[0:4]], datatl[datatl.columns[0:4]], datarw[datarw.columns[0:4]], datals[datals.columns[0:4]], datazy[datazy.columns[0:4]], datasc[datasc.columns[0:4]]], axis = 0)
data = data.sort_values('日期').reset_index(drop = True)
print(len(data['活动类型'].unique()))
print(data.shape)
data.head()

29
(492504, 4)


,日期,活动类型,用户id,课程id
0,2021-03-01 07:48,已签,28412052,1764396
1,2021-03-01 07:48,已签,28412076,1764396
2,2021-03-01 07:48,已签,28412088,1764396
3,2021-03-01 07:48,已签,28412041,1764396
4,2021-03-01 07:48,已签,28412046,1764396


## 2.数据增广 —— 活动类型的值进行自定义映射
活动类型一共有26种，如果直接作为特征维度会导致数据非常稀疏，部分特征有效样本数非常少，所以我们将特征维度代表意义类似的用自定义特征统一，并将样本数组合增大有效样本数，最终的特征维度选择如下：

'已签'、'AttendLog'、'公假'、'代签'——'出勤'

'签到已过期'、'事假'、'病假'——'缺勤'


 '查阅作业分数'、'查阅作业答案'、'查阅作业评价'' 查阅考试成绩'、'查阅试卷答案'、'查阅测验答案'、'查阅测验分数'、'访问课程'、'登录访问'——'自主学习'
 
 
 '批阅互评作业'、'VoteLog'、'PreampAnswerLog''ChooseSomeoneLog'——'互动'
 
 'WorkLog'、'测验'——'测试’
 
 'TaskLog'——'小组合作'
 
 '讨论'、'回复讨论'——'讨论'
 
 '提交作业'、'完成作业'——'完成作业'
 
 '未测'——'未测'
 
 '完成任务点'——'完成任务点'

**'出勤'、 '测试'、 '自主学习'、 '讨论'、 '缺勤'、 '互动'、 '完成作业'、 '完成任务点'、 '未测'、 '小组合作'**

In [42]:
types = list(data['活动类型'].unique())
typevals = ['出勤', '测试', '自主学习', '讨论', '自主学习', '出勤', '缺勤', '互动', '测试', '完成作业', '自主学习', '自主学习', '互动', '完成任务点', '自主学习', '自主学习', '自主学习', '自主学习', '自主学习', '互动', '未测', '完成作业', '讨论', '出勤', '缺勤', '缺勤', '小组合作', '互动', '出勤']
typesets = dict(zip(types, typevals))
cdata = data.copy()
cdata['活动类型'] = cdata['活动类型'].map(typesets)
cdata.reset_index(drop = True, inplace = True)
print(cdata.shape)
print(cdata['活动类型'].unique())
cdata.head()

(492504, 4)
['出勤' '测试' '自主学习' '讨论' '缺勤' '互动' '完成作业' '完成任务点' '未测' '小组合作']


,日期,活动类型,用户id,课程id
0,2021-03-01 07:48,出勤,28412052,1764396
1,2021-03-01 07:48,出勤,28412076,1764396
2,2021-03-01 07:48,出勤,28412088,1764396
3,2021-03-01 07:48,出勤,28412041,1764396
4,2021-03-01 07:48,出勤,28412046,1764396


将汇总增广后的数据集保存先进行关联规则的分析

In [6]:
# cdata.to_csv('./input/STAR_关联规则.csv', index = False, encoding = 'utf_8_sig')

## 3.离散特征——独热编码

In [43]:
from sklearn.preprocessing import  OneHotEncoder
# onehot编码
da = cdata[['活动类型']]
ohe = OneHotEncoder(sparse = False)
ar = ohe.fit_transform(da)
print(ohe.get_feature_names_out())
cols = ['互动', '出勤', '完成任务点', '完成作业', '小组合作', '未测', '测试', '缺勤', '自主学习', '讨论']
# cols = ['互动', '出勤', '完成任务点', '完成作业', '小组合作', '测试', '缺勤', '自主学习', '讨论']
# df_new = pd.DataFrame(ar, columns = ohe.get_feature_names_out())# 将数据和标题整合
df_new = pd.DataFrame(ar, columns = cols)
datas = pd.concat([cdata[['日期', *cdata.columns[2:]]], df_new], axis = 1)
print(datas.shape)
datas.head()

['活动类型_互动' '活动类型_出勤' '活动类型_完成任务点' '活动类型_完成作业' '活动类型_小组合作' '活动类型_未测'
 '活动类型_测试' '活动类型_缺勤' '活动类型_自主学习' '活动类型_讨论']
(492504, 13)


,日期,用户id,课程id,互动,出勤,完成任务点,完成作业,小组合作,未测,测试,缺勤,自主学习,讨论
0,2021-03-01 07:48,28412052,1764396,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-03-01 07:48,28412076,1764396,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-03-01 07:48,28412088,1764396,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-03-01 07:48,28412041,1764396,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-03-01 07:48,28412046,1764396,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 特征工程——不保留时间特征，按照学生与课程对应关系汇总学生操作流水，扩大每个特征的样本值

## 4.统计学生不同课程中的各类学习行为次数

In [55]:
def countlogs(data, newdata, kk = 0):
    '''
    data: 原数据集
    ds: 新数据集
    kk: 计数
    return ds
    '''
    ds = newdata
    for i in data['课程id'].unique():
        da = data[data['课程id'] == i]
        for ns in da.groupby('用户id')[da.columns[3:]]:
            if ns[1].shape[0] > 1:
                ds.loc[kk, :] = [ns[0], i, *list(ns[1].sum())]
                kk += 1
            else:
                ds.loc[kk, :] = [ns[0], i, *ns[1].values[0]]
                kk += 1
    print(ds.shape)
    return ds

ds = countlogs(datas, pd.DataFrame(columns = datas.columns[1:]))

(55972, 12)


In [56]:
ds_1 = ds.copy()
ds_1.head()

,用户id,课程id,互动,出勤,完成任务点,完成作业,小组合作,未测,测试,缺勤,自主学习,讨论
0,28412037,1764396,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,28412039,1764396,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,28412040,1764396,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28412041,1764396,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,28412042,1764396,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [44]:
datasc_1 = datasc[['日期', *datasc.columns[2:4], '分数']].rename(columns = {'分数':'测试成绩'})
datazy_1 = datazy[['日期', *datazy.columns[2:4], '成绩']].rename(columns = {'成绩':'作业成绩'})
datasczy = pd.concat([datazy_1, datasc_1], axis = 0)
datasczy.shape

(37986, 5)

**经过以上操作，我们的特征从一开始的2维扩充13维**

## 5.增加课程作业平均成绩列
新增一列，每名学生每门课程的作业平均成绩

In [57]:
for i in datazy.groupby(['课程id', '用户id'])['成绩']:
    if ds_1[(ds_1['课程id'] == i[0][0]) & (ds_1['用户id'] == i[0][1])].shape[0] > 0:
        ds_1.loc[(ds_1['课程id'] == i[0][0]) & (ds_1['用户id'] == i[0][1]), '作业平均成绩'] = i[1].mean().round(2)
    else:
        print(i)

In [58]:
ds_1[(ds_1['课程id'] == 222602451) & (ds_1['用户id'] == 51466294)]

,用户id,课程id,互动,出勤,完成任务点,完成作业,小组合作,未测,测试,缺勤,自主学习,讨论,作业平均成绩
37905,51466294,222602451,2.0,19.0,54.0,14.0,2.0,0.0,0.0,7.0,285.0,0.0,74.12


## 6.增加课堂测试平均成绩列
新增一列，每名学生在每门课程的课堂测试平均成绩

In [59]:
for i in datasc.groupby(['课程id', '用户id'])['分数']:
    if ds_1[(ds_1['课程id'] == i[0][0]) & (ds_1['用户id'] == i[0][1])].shape[0] > 0:
        ds_1.loc[(ds_1['课程id'] == i[0][0]) & (ds_1['用户id'] == i[0][1]), '测试平均成绩'] = i[1].mean().round(2)
    else:
        if i[1].mean() != 0:
            print(i)

In [60]:
ds_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55972 entries, 0 to 55971
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   用户id    55972 non-null  object 
 1   课程id    55972 non-null  object 
 2   互动      55972 non-null  object 
 3   出勤      55972 non-null  object 
 4   完成任务点   55972 non-null  object 
 5   完成作业    55972 non-null  object 
 6   小组合作    55972 non-null  object 
 7   未测      55972 non-null  object 
 8   测试      55972 non-null  object 
 9   缺勤      55972 non-null  object 
 10  自主学习    55972 non-null  object 
 11  讨论      55972 non-null  object 
 12  作业平均成绩  868 non-null    float64
 13  测试平均成绩  6483 non-null   float64
dtypes: float64(2), object(12)
memory usage: 8.4+ MB


### 补充新增两列的缺失值——用0补充
因为这里的成绩是连续值，所以无法用取众数方法处理，我们采用均值填充

In [61]:
ds_1['作业平均成绩'].mean(), ds_1['测试平均成绩'].mean()

(73.37455069124424, 66.82905445010027)

In [62]:
ds_1['作业平均成绩'] = ds_1['作业平均成绩'].fillna(ds_1['作业平均成绩'].mean()) # 均值填充
ds_1['测试平均成绩'] = ds_1['测试平均成绩'].fillna(ds_1['测试平均成绩'].mean()) # 均值填充

In [63]:
ds_1['作业平均成绩'] = ds_1['作业平均成绩'].round(2)
ds_1['测试平均成绩'] = ds_1['测试平均成绩'].round(2)

In [64]:
ds_1.head()

,用户id,课程id,互动,出勤,完成任务点,完成作业,小组合作,未测,测试,缺勤,自主学习,讨论,作业平均成绩,测试平均成绩
0,28412037,1764396,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.37,66.83
1,28412039,1764396,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.37,66.83
2,28412040,1764396,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.37,66.83
3,28412041,1764396,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.37,66.83
4,28412042,1764396,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.37,66.83


## 7.设置label——综合成绩
### 不补充 label 的缺省值，而是直接划分数据集

In [45]:
from sklearn.ensemble import RandomForestRegressor
def getrfrlabel(cdata, x, y):
    '''
    cdata: 数据集
    x: 特征变量
    y: 标签
    return: 标签
    '''
    X_train = cdata[~cdata[y].isna()][cols]
    y_train = cdata[~cdata[y].isna()][y]
    X_test = cdata[cdata[y].isna()][cols]

    # 用随机森林回归来填补缺失值
    rfc = RandomForestRegressor(n_estimators = 100)
    rfc = rfc.fit(X_train, y_train)
    predict = rfc.predict(X_test)    
    return predict

### 整理学生综合成绩表

In [67]:
cid = ds_1.drop_duplicates(['用户id', '课程id']).groupby(['用户id', '课程id'])['互动'].count().index.to_list()
datascore = dfscore[['用户id', '课程id', '综合成绩']]
print(datascore.shape)
datascore.head()

(28838, 3)


,用户id,课程id,综合成绩
0,51465331,223018597,88.09
1,67473135,223018597,100.00
2,69984054,223018597,92.42
3,111943347,223018597,88.68
4,111944035,223018597,94.25


In [68]:
for k in dfscore[['用户id', '课程id']].drop_duplicates().groupby(['用户id', '课程id']):
    if k[0] not in cid:
        datascore.drop(datascore[(datascore['用户id'] == k[0][0]) & (datascore['课程id'] == k[0][1])].index, inplace = True)

C:\Users\SUGA\AppData\Local\Temp\ipykernel_21564\894643865.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  datascore.drop(datascore[(datascore['用户id'] == k[0][0]) & (datascore['课程id'] == k[0][1])].index, inplace = True)


### 特征工程——添加标签

In [69]:
ds_2 = pd.merge(ds_1, datascore, on = ['用户id', '课程id'], how = 'outer')
print(ds_1.shape, ds_2.shape)
ds_2 = ds_2.drop_duplicates(ds_2.columns[0:-1])
ds_2.head()

(55972, 14) (58905, 15)


,用户id,课程id,互动,出勤,完成任务点,完成作业,小组合作,未测,测试,缺勤,自主学习,讨论,作业平均成绩,测试平均成绩,综合成绩
0,28412037,1764396,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.37,66.83,NaN
1,28412039,1764396,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.37,66.83,NaN
2,28412040,1764396,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.37,66.83,NaN
3,28412041,1764396,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.37,66.83,NaN
4,28412042,1764396,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73.37,66.83,NaN


In [235]:
# 填充缺省值
# ds_2.loc[ds_2['综合成绩'].isna(), '综合成绩'] = getrfrlabel(ds_2, ds_2.columns[2:-1], '综合成绩')
# ds_2['综合成绩'] = ds_2['综合成绩'].round(2)
# print(ds_2.shape)
# ds_2.head()

(55961, 12)


,用户id,课程id,互动,出勤,完成任务点,完成作业,小组合作,测试,缺勤,自主学习,讨论,综合成绩
0,28412037,1764396,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.66
1,28412039,1764396,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.66
2,28412040,1764396,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.66
3,28412041,1764396,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.66
4,28412042,1764396,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,78.66


因为综合成绩有缺省值，缺失率占总的样本数59%左右，所以我们这边直接进行训练集划分，将Label非空的的样本数据作为训练集共22834条样本数据，Label为空的样本数据作为测试集共33138条样本数据，所以训练集与测试集的比例是4:6。

In [70]:
print(ds_2[~ds_2['综合成绩'].isna()].shape[0], ds_2[ds_2['综合成绩'].isna()].shape[0])
ds_2[~ds_2['综合成绩'].isna()].shape[0]/ds_2[ds_2['综合成绩'].isna()].shape[0], ds_2[ds_2['综合成绩'].isna()].shape[0]/ds_2.shape[0]

22834 33138


(0.6890578791719476, 0.5920460230115058)

## 8.新增在线时长的特征维度和姓名列

In [72]:
datastu = pd.read_csv('E:/比赛/data/1.学生名单.csv')
print(datastu.shape)
datastu.head()

(34246, 4)


,用户id,姓名,学号,在线时长（秒）
0,51465331,丁屹中,20161003949,7047
1,111944914,娜迪热·艾尔肯,20191004561,209251
2,111946468,何倩,20191004398,207885
3,147372556,熊锐,20201001447,334696
4,147374240,郑玉洁,20201001777,118632


In [73]:
sss = pd.merge(ds_2, datastu[['用户id', '姓名', '在线时长（秒）']], on = '用户id')
ss = sss[['姓名', *sss.columns[0:-3], '在线时长（秒）', '综合成绩']]

In [74]:
co = ss.columns
cos = ['name', 'personid', 'courseid', 'interact', 'arrive', 'finishtask', 'finishwork', 'group', 'notest', 'test', 'notarrived','selflearning', 'talk', 'workscore', 'testscore', 'duration(s)', 'score']
newcos = dict(zip(co, cos))

In [78]:
ss.columns = cos
ss = ss.sort_values('selflearning', ascending = False).reset_index(drop = True)
print(ss.shape)
ss.head(10)

(55962, 17)


,name,personid,courseid,interact,arrive,finishtask,finishwork,group,notest,test,notarrived,selflearning,talk,workscore,testscore,duration(s),score
0,夏春秋,98301060,222602451,1.0,21.0,58.0,14.0,2.0,0.0,0.0,9.0,982.0,1.0,92.29,66.83,1139355,82.41
1,侯宇凡,111947743,222602451,3.0,23.0,58.0,14.0,2.0,0.0,0.0,9.0,474.0,1.0,93.51,66.83,407798,82.62
2,刘安,98301020,222602451,1.0,23.0,58.0,14.0,2.0,0.0,0.0,9.0,470.0,1.0,85.49,66.83,232641,79.90
3,林杰,111946630,222820410,0.0,21.0,34.0,18.0,0.0,0.0,5.0,1.0,466.0,0.0,97.94,96.40,492055,56.72
4,李启睿,98289131,222602451,3.0,23.0,58.0,14.0,2.0,0.0,0.0,9.0,412.0,1.0,76.89,66.83,264115,77.16
5,高天弘,111945472,222820410,0.0,21.0,88.0,22.0,0.0,0.0,5.0,1.0,407.0,0.0,98.69,98.00,207890,58.91
6,詹吉超,111948894,222820410,0.0,21.0,6.0,22.0,0.0,0.0,5.0,1.0,405.0,0.0,98.78,94.00,324882,55.06
7,许倩,111948685,222820410,0.0,21.0,88.0,21.0,0.0,0.0,5.0,1.0,403.0,0.0,89.82,96.40,511577,55.09
8,张泽昊,111947311,222820410,0.0,14.0,28.0,12.0,0.0,0.0,5.0,0.0,386.0,0.0,94.44,95.00,314773,92.22
9,王煜,111949227,222602451,2.0,21.0,58.0,14.0,2.0,0.0,0.0,9.0,378.0,0.0,87.41,66.83,316029,80.72


## 8.列数据类型转换

In [85]:
# 特征工程——第一类
for col in ss.columns:
    if col not in ['name', 'personid', 'courseid']:
        if col in ['workscore', 'testscore', 'score']:
            ss[col] = ss[col].astype(float)
        else:
            ss[col] = ss[col].astype(int)
    else:
        if col == 'name':
            ss[col] = ss[col].astype(object) 
        else:
            ss[col] = (ss[col].astype(int)).astype(object)

In [86]:
print(ss.shape)
ss.head()

(55962, 17)


,name,personid,courseid,interact,arrive,finishtask,finishwork,group,notest,test,notarrived,selflearning,talk,workscore,testscore,duration(s),score
0,夏春秋,98301060,222602451,1,21,58,14,2,0,0,9,982,1,92.29,66.83,1139355,82.41
1,侯宇凡,111947743,222602451,3,23,58,14,2,0,0,9,474,1,93.51,66.83,407798,82.62
2,刘安,98301020,222602451,1,23,58,14,2,0,0,9,470,1,85.49,66.83,232641,79.90
3,林杰,111946630,222820410,0,21,34,18,0,0,5,1,466,0,97.94,96.40,492055,56.72
4,李启睿,98289131,222602451,3,23,58,14,2,0,0,9,412,1,76.89,66.83,264115,77.16


## 展示数据集

In [88]:
ss.describe()

,interact,arrive,finishtask,finishwork,group,notest,test,notarrived,selflearning,talk,workscore,testscore,duration(s),score
count,55962.000000,55962.000000,55962.000000,55962.000000,55962.000000,55962.000000,55962.000000,55962.000000,55962.000000,55962.000000,55962.000000,55962.000000,5.596200e+04,22834.000000
mean,0.010275,6.178925,0.096708,0.108306,0.002609,0.000536,0.648601,0.020335,1.687038,0.046746,73.370071,66.829890,1.178298e+05,75.473660
std,0.362487,6.247636,2.307520,1.102043,0.072188,0.028666,3.250458,0.316475,13.882927,3.075911,4.544044,10.133594,2.017724e+05,7.676716
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.800000e+02,0.000000
25%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,73.370000,66.830000,4.069000e+03,70.000000
50%,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,73.370000,66.830000,4.131800e+04,74.000000
75%,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,73.370000,66.830000,1.402860e+05,80.000000
max,33.000000,45.000000,88.000000,24.000000,2.000000,3.000000,47.000000,9.000000,982.000000,501.000000,100.000000,100.000000,3.826430e+06,100.000000


In [20]:
# import pandas as pd
# c = pd.read_csv('./input/STAR_1.csv')
c.describe()['duration(s)']

count    5.596200e+04
mean     1.178298e+05
std      2.017724e+05
min      1.800000e+02
25%      4.069000e+03
50%      4.131800e+04
75%      1.402860e+05
max      3.826430e+06
Name: duration(s), dtype: float64

## 保存数据

In [90]:
ss.to_csv('./input/STAR_1.csv', index = False, encoding = 'utf_8_sig')

In [91]:
def train_test(data):
    train = data[~data.score.isna()].reset_index(drop = True)
    test = data[data.score.isna()].reset_index(drop = True)
    return train, test

In [92]:
train, test = train_test(ss)
train.to_csv('./output/STAR_1_train.csv', index = False, encoding = 'utf_8_sig')
test.to_csv('./output/STAR_1_test.csv', index = False, encoding = 'utf_8_sig')